# Dataframe Creation for E11 CH Composition Study

Tamar Ervin
Jan 4, 2024

In [2]:
import glob
import pyspedas
from pyspedas import time_string, time_double
from pytplot import tplot, get_data, cdf_to_tplot, store_data
import astrospice
import sunpy 
import sys, os
import datetime
import numpy as np
sys.path.append(os.path.realpath(''))
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt

import tools.utilities as utils
import tools.sigma as sigma
import tools.psp_funcs as psp_funcs
import tools.pfss_funcs as pfss_funcs


import sunpy.coordinates as scoords
from scipy.interpolate import interp1d
from plasmapy.formulary import beta, magnetic_pressure, thermal_pressure, ion_sound_speed
from astropy.constants import k_B
# import heliopy.data.spice as spicedata
# import heliopy.spice as spice
import scipy.constants as con


import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import animation, rc
from IPython.display import HTML
from astropy.coordinates import SkyCoord

import astropy.constants as const
import astropy.units as u
from ntpath import basename
import numpy as np
from urllib.request import urlretrieve
import os 

rc('animation',html='html5')


RES_DIR = os.path.realpath('results')
for sc in ['psp','solar orbiter'] : kernels = astrospice.registry.get_kernels(sc,'predict') 



Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

05-Jan-24 17:24:18: /Users/tamarervin/miniconda3/envs/analysis/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),



Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

# Data Download

In [5]:
time_range = ['2022-02-23/00:00', '2022-03-01/00:00']
enc = 'E11'
edens = 'ENC11_QTN_Electron_Density.csv'

In [6]:
# FIELDS
fields_vars = pyspedas.psp.fields(trange=time_range, time_clip=True, datatype='mag_RTN_4_Sa_per_Cyc')

# RFS
rfs_vars = pyspedas.psp.fields(trange=time_range, time_clip=True, level='l3', datatype='rfs_lfr')

# SPAN-Ion - Proton Moments
pvars = pyspedas.psp.spi(trange=time_range, datatype='sf00_l3_mom', 
                            level='l3', time_clip=True)

# SPAN-Ion - Alpha Particle Moments
avars = pyspedas.psp.spi(trange=time_range, datatype='sf0a_l3_mom', 
                            level='l3', time_clip=True)

# SPAN-Electron - Electron Moments
spe_vars = pyspedas.psp.spe(trange=time_range, level='l2', time_clip=True)

05-Jan-24 17:24:29: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn_4_per_cycle/2022/
05-Jan-24 17:24:30: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn_4_per_cycle/2022/psp_fld_l2_mag_rtn_4_sa_per_cyc_20220223_v02.cdf to psp_data/fields/l2/mag_rtn_4_per_cycle/2022/psp_fld_l2_mag_rtn_4_sa_per_cyc_20220223_v02.cdf
05-Jan-24 17:24:31: Download complete: psp_data/fields/l2/mag_rtn_4_per_cycle/2022/psp_fld_l2_mag_rtn_4_sa_per_cyc_20220223_v02.cdf
05-Jan-24 17:24:31: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn_4_per_cycle/2022/psp_fld_l2_mag_rtn_4_sa_per_cyc_20220224_v02.cdf to psp_data/fields/l2/mag_rtn_4_per_cycle/2022/psp_fld_l2_mag_rtn_4_sa_per_cyc_20220224_v02.cdf
05-Jan-24 17:24:32: Download complete: psp_data/fields/l2/mag_rtn_4_per_cycle/2022/psp_fld_l2_mag_rtn_4_sa_per_cyc_20220224_v02.cdf
05-Jan-24 17:24:32: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn_4_per_cycle/2022/psp

Using LEVEL=L3


05-Jan-24 17:25:02: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2022/psp_swp_spi_sf00_l3_mom_20220223_v04.cdf to psp_data/sweap/spi/l3/spi_sf00_l3_mom/2022/psp_swp_spi_sf00_l3_mom_20220223_v04.cdf
05-Jan-24 17:25:03: Download complete: psp_data/sweap/spi/l3/spi_sf00_l3_mom/2022/psp_swp_spi_sf00_l3_mom_20220223_v04.cdf
05-Jan-24 17:25:04: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2022/psp_swp_spi_sf00_l3_mom_20220224_v04.cdf to psp_data/sweap/spi/l3/spi_sf00_l3_mom/2022/psp_swp_spi_sf00_l3_mom_20220224_v04.cdf
05-Jan-24 17:25:06: Download complete: psp_data/sweap/spi/l3/spi_sf00_l3_mom/2022/psp_swp_spi_sf00_l3_mom_20220224_v04.cdf
05-Jan-24 17:25:06: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2022/psp_swp_spi_sf00_l3_mom_20220225_v04.cdf to psp_data/sweap/spi/l3/spi_sf00_l3_mom/2022/psp_swp_spi_sf00_l3_mom_20220225_v04.cdf
05-Jan-24 17:25:07: Download complete: psp_data/swea

Using LEVEL=L3


05-Jan-24 17:25:19: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf0a_l3_mom/2022/psp_swp_spi_sf0a_l3_mom_20220223_v04.cdf to psp_data/sweap/spi/l3/spi_sf0a_l3_mom/2022/psp_swp_spi_sf0a_l3_mom_20220223_v04.cdf
05-Jan-24 17:25:20: Download complete: psp_data/sweap/spi/l3/spi_sf0a_l3_mom/2022/psp_swp_spi_sf0a_l3_mom_20220223_v04.cdf
05-Jan-24 17:25:20: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf0a_l3_mom/2022/psp_swp_spi_sf0a_l3_mom_20220224_v04.cdf to psp_data/sweap/spi/l3/spi_sf0a_l3_mom/2022/psp_swp_spi_sf0a_l3_mom_20220224_v04.cdf
05-Jan-24 17:25:22: Download complete: psp_data/sweap/spi/l3/spi_sf0a_l3_mom/2022/psp_swp_spi_sf0a_l3_mom_20220224_v04.cdf
05-Jan-24 17:25:22: Downloading https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf0a_l3_mom/2022/psp_swp_spi_sf0a_l3_mom_20220225_v04.cdf to psp_data/sweap/spi/l3/spi_sf0a_l3_mom/2022/psp_swp_spi_sf0a_l3_mom_20220225_v04.cdf
05-Jan-24 17:25:25: Download complete: psp_data/swea

# PARKER

In [9]:
# check if folder exists
if not os.path.exists(RES_DIR):
    os.makedirs(RES_DIR)
    print(f"Folder '{RES_DIR}' created.")
else:
    print(f"Folder '{RES_DIR}' already exists.")

Folder '/Users/tamarervin/publications/e11_ch/results' created.


### FIELDS

In [10]:
files = glob.glob(os.path.join(os.path.realpath(os.path.join('psp_data', 'fields/l2/mag_rtn_4_per_cycle/2023')), "*"), recursive=True)
vars = cdf_to_tplot(files)
dt = get_data('psp_fld_l2_mag_RTN_4_Sa_per_Cyc')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'Br': dt.y[:, 0], 'Bt': dt.y[:, 1], 'Bn': dt.y[:, 2]}
fields = pd.DataFrame(data=rd)
fields.to_csv(os.path.join(RES_DIR, 'fields.csv'))

OSError: [Errno 28] No space left on device

In [ ]:
tplot(['psp_fld_l2_mag_RTN_4_Sa_per_Cyc'])

### PROTONS

In [ ]:
path = glob.glob(os.path.join(os.path.realpath(os.path.join('psp_data', 'sweap/spi/l3/spi_sf00_l3_mom')), "*"), recursive=True)
files = glob.glob(os.path.join(path[0], '*'))

vars = cdf_to_tplot(files)
dt = get_data('VEL_RTN_SUN')
dt2 = get_data('DENS')
dt3 = get_data('TEMP')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'vr': np.abs(dt.y[:, 0]), 'vt': dt.y[:, 1], 'vn': dt.y[:, 2], 'Np': dt2.y, 'Tp': dt3.y}
protons = pd.DataFrame(data=rd)

### ADD ANGLE
vx, vy, vz = [get_data('VEL_SC').y[:, i] for i in np.arange(0, 3)]
mx, my, mz = [get_data('MAGF_INST').y[:, i] for i in np.arange(0, 3)]
vdotb = vx*mx + vy*my + vz*mz
v = np.sqrt(vx**2 + vy**2 + vz**2)
b = np.sqrt(mx**2 + my**2 + mz**2)
angle_vb = np.arccos(vdotb/(v*b))

protons['angle_vb'] = angle_vb

protons.to_csv(os.path.join(RES_DIR, 'protons.csv'))

In [ ]:
tplot(['VEL_INST',
 'VEL_SC',
 'VEL_RTN_SUN',
 'SC_VEL_RTN_SUN'])
vars


In [ ]:
tplot(['DENS', 'VEL_RTN_SUN', 'TEMP', 'EFLUX_VS_THETA', 'EFLUX_VS_PHI'])

### ALPHAS

In [ ]:
path = glob.glob(os.path.join(os.path.realpath(os.path.join('psp_data', 'sweap/spi/l3/spi_sf0a_l3_mom')), "*"), recursive=True)
files = glob.glob(os.path.join(path[0], '*'))

vars = cdf_to_tplot(files)
dt = get_data('VEL_RTN_SUN')
dt2 = get_data('DENS')
dt3 = get_data('TEMP')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'vra': dt.y[:, 0], 'vta': dt.y[:, 1], 'vna': dt.y[:, 2], 'Na': dt2.y, 'Ta': dt3.y}
alphas = pd.DataFrame(data=rd)
alphas.to_csv(os.path.join(RES_DIR, 'alphas.csv'))

In [ ]:
tplot(['DENS', 'VEL_RTN_SUN', 'TEMP', 'EFLUX_VS_THETA', 'EFLUX_VS_PHI'])

### ELECTRONS

In [ ]:
dt = pd.read_csv(os.path.realpath(edens))
date_obj = [datetime.datetime.strptime(d, '%Y-%m-%d/%H:%M:%S') for d in dt.Time]

rd = {'Time': date_obj, 'Ne': dt.N_QTN}
dfe = pd.DataFrame(data=rd, index=None)
# dfe = dfe.set_index(dfe.Time)
dfe.to_csv(os.path.join(RES_DIR, 'electrons.csv'))

### COMBINE DATASETS

In [ ]:
RESULTS = '/Users/tamarervin/e11_ch_results/'
RESULTS = os.path.realpath('e11_ch_results')

In [ ]:
df = pd.read_csv(os.path.join(RESULTS, 'protons.csv'), index_col=None)
dfe = pd.read_csv(os.path.join(RESULTS, 'electrons.csv'), index_col=None)
# dfet = pd.read_csv(os.path.join(RESULTS, 'electron_temp.csv'), index_col=None)
dfalp = pd.read_csv(os.path.join(RESULTS, 'alphas.csv'), index_col=None)
dfmag = pd.read_csv(os.path.join(RESULTS, 'fields.csv'), index_col=None)
# df, dfe, dfet, dfalp = [dd.drop(['Unnamed: 0'], axis=1) for dd in [df, dfe, dfet, dfalp]]
dfmag = dfmag.drop(['Time.1'], axis=1)
df['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in df.Time]
dfe['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dfe.Time]
dfet['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dfet.Time]
dfalp['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in dfalp.Time]
dfmag['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in dfmag.Time]


In [ ]:
ff = pd.merge_asof(df, dfalp, on='Time', direction='backward')
ff = pd.merge_asof(ff, dfe, on='Time', direction='backward')
ff = pd.merge_asof(ff, dfet, on='Time', direction='backward')
ff = pd.merge_asof(ff, dfmag, on='Time', direction='backward')
ff = ff.set_index('Time')
ff = ff[np.logical_and(ff.Tp > 0, ff.Ta > 0)].copy()
ff

In [ ]:
# Calculate time differences between consecutive datetime values
from datetime import timedelta
datetime_list = ff.index.to_list()
time_diff_list = [datetime_list[i + 1] - datetime_list[i] for i in range(len(datetime_list) - 1)]

# Calculate the average time difference
average_time_difference = sum(time_diff_list, timedelta()) / len(time_diff_list)
print(average_time_difference)

# Specify the total time period (e.g., 10 minutes)
total_time_period = timedelta(minutes=20)

# Calculate the number of points
number_of_points = total_time_period / average_time_difference

print(f"Number of Points in {total_time_period}: {int(number_of_points)}")


In [ ]:
### ----- CALCULATIONS ----- ###
### Vap
ff['B'] = np.sqrt(ff.Br**2 + ff.Bt**2 + ff.Bn**2)
cost = np.abs(ff.Br/ff.B)
ff['vap'] = (ff.vra - ff.vr)/cost

### Ahe
ff['Ahe'] = ff.Na/ff.Np

### TUBULENCE
ff['use_dens'] = ff.Np
ff['sigmac'],ff['sigmar'], ff['vA'], ff['Zp'],ff['Zm'], ff['deltav'], ff['deltab'] = sigma.calc_sigma(ff, num=314)
ff = ff.drop(['use_dens'], axis=1)
ff['diff'] = np.abs(ff.vap)/ff.vA

### MACH NUMBER
ff['MA'] = ff.vr / ff.vA
sound_speed = ion_sound_speed(
    T_e=np.array(ff.Te)*u.eV/k_B,
    T_i=np.array(ff.Tp)*u.eV/k_B,
    # n_e=np.array(ff.Ne)/(u.cm**3),
    # k=k_2,
    ion='p',
    gamma_e=1,
    gamma_i=3,
).to(u.km/u.s).value
ff['MS'] = ff.vr / sound_speed
ff['MMS'] = ff.vr / np.sqrt(sound_speed**2 + ff.vA**2)

### BETA
ff['beta'] = beta(np.array(ff.Tp)*u.eV, np.array(ff.Np)/(u.cm*u.cm*u.cm), np.array(ff.B)*u.nT).value
ff['betae'] = beta(np.array(ff.Te)*u.eV, np.array(ff.Ne)/(u.cm*u.cm*u.cm), np.array(ff.B)*u.nT).value

# ### MAGNETIC PRESSURE
ff['Pm'] = magnetic_pressure(np.array(ff.B)*u.nT).value

### PROTON PRESSURE
ff['Pp'] = thermal_pressure(np.array(ff.Tp)*u.eV, np.array(ff.Np)/u.cm**3).value

### ALPHA PRESSURE
ff['Pa'] = thermal_pressure(np.array(ff.Ta)*u.eV, np.array(ff.Na)/u.cm**3).value

### ELECTRON PRESSURE
ff['Pe'] = thermal_pressure(np.array(ff.Te)*u.eV, np.array(ff.Ne)/u.cm**3).value

### PARKER DF
parker = ff.copy()
parker

In [ ]:
### FLAG DATA

# check if there is any alpha data
all_nan = parker['Ahe'].isna().all()
parker['flag'] = np.ones(len(parker.Ahe))
if all_nan:
    parker['flag'] = np.zeros(len(parker.Ahe))
else:
    # Flag instances as '0' if all three conditions apply
    flag_condition_1 = (np.abs(parker.Np - parker.Ne) / parker.Ne) <= 0.5
    flag_condition_2 = (parker.Ahe) <= 0.2
    flag_condition_3 = (np.abs(parker.vap) / parker.vA) <= 2
    flag_condition_4 = parker.Te < 200 

    flag_0 = np.logical_and.reduce([flag_condition_1, flag_condition_2, flag_condition_3, flag_condition_4])

parker['flag'][flag_0] = 0


In [ ]:
### Create SkyCoord for PSP in the inertial (J2000) frame
tt = pd.to_datetime(ff.index.to_list())

psp_inertial = astrospice.generate_coords(
    'SOLAR PROBE PLUS', tt
)

### Transform to solar co-rotating frame 
psp_carrington = psp_inertial.transform_to(
    scoords.HeliographicCarrington(observer="self")
)

# projection
ts_common = np.array([dt.timestamp() for dt in tt])
psp_vr_ts = [int(dt.timestamp()) for dt in tt]
psp_vr_common = interp1d(psp_vr_ts,ff.vr,bounds_error=False)(ts_common)*u.km/u.s
psp_at_source_surface = psp_funcs.ballistically_project(psp_carrington,vr_arr=psp_vr_common, r_inner=2.0*u.R_sun)


In [ ]:
### ADD POSITION INFORMAITON AND SAVE
parker['lon'] = psp_carrington.lon.value
parker['lat'] = psp_carrington.lat.value
parker['rAU'] = psp_carrington.radius.to(u.AU).value
parker['sslon'] = psp_at_source_surface.lon.value
parker['sslat'] = psp_at_source_surface.lat.value
parker['ssrAU'] = psp_at_source_surface.radius.to(u.AU).value
parker['NpR2'] = parker.Np * (parker.rAU ** 2)
parker['NeR2'] = parker.Ne * (parker.rAU ** 2)
parker['BrR2'] = parker.Br * (parker.rAU ** 2)
parker.to_csv(os.path.join(RESULTS, 'parker.csv'))

# ORBITER

In [ ]:
files = glob.glob(os.path.join(os.path.realpath(os.path.join('solar_orbiter_data', 'mag/L2/2023')), "*"), recursive=True)

vars = cdf_to_tplot(files)
dt = get_data('B_RTN')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'Br': dt.y[:, 0], 'Bt': dt.y[:, 1], 'Bn': dt.y[:, 2]}
mag = pd.DataFrame(data=rd)
# mag = df.set_index(df.Time)
mag.to_csv(os.path.join(RES_DIR, 'mag.csv'))

In [ ]:
files = glob.glob(os.path.join(os.path.realpath(os.path.join('solar_orbiter_data', 'swa/L2/2023')), "*"), recursive=True)

vars = cdf_to_tplot(files)
dt = get_data('V_RTN')
dt2 = get_data('N')
dt3 = get_data('T')
date_obj = [datetime.datetime.strptime(time_string(d), '%Y-%m-%d %H:%M:%S.%f') for d in dt.times]

rd = {'Time': date_obj, 'vr': dt.y[:, 0], 'vt': dt.y[:, 1], 'vn': dt.y[:, 2], 'Np': dt2.y, 'Tp': dt3.y}
swa = pd.DataFrame(data=rd)
# swa = df.set_index(df.Time)
swa.to_csv(os.path.join(RES_DIR, 'swa.csv'))

In [ ]:
tplot(['N', 'V_RTN', 'B_RTN', 'T'])


In [ ]:
ff = pd.merge_asof(swa, mag, on='Time', direction='backward')
ff = ff.set_index('Time')
ff

In [ ]:
orbiter = pd.read_csv('results/orbiter.csv')
orbiter['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in orbiter.Time]
orbiter

In [ ]:
### Create SkyCoord for PSP in the inertial (J2000) frame
ff = swa
tt = pd.to_datetime(ff.index.to_list())
kernels =  astrospice.registry.get_kernels('solar orbiter','predict') 
solo_inertial = astrospice.generate_coords(
    'SOLAR ORBITER', tt

)

### Transform to solar co-rotating frame 
solo_carrington = solo_inertial.transform_to(
    scoords.HeliographicCarrington(observer="self")
)

# projection
ts_common = np.array([dt.timestamp() for dt in tt])
solo_vr_ts = [int(dt.timestamp()) for dt in tt]
solo_vr_common = interp1d(solo_vr_ts,ff.vr,bounds_error=False)(ts_common)*u.km/u.s
solo_at_source_surface = psp_funcs.ballistically_project(solo_carrington,vr_arr=solo_vr_common, r_inner=2.0*u.R_sun)


In [ ]:
### ----- CALCULATIONS ----- ###
ff = orbiter
### Vap
ff['B'] = np.sqrt(ff.Br**2 + ff.Bt**2 + ff.Bn**2)

### TUBULENCE
ff['use_dens'] = ff.Np
ff['sigmac'],ff['sigmar'], ff['vA'], ff['Zp'],ff['Zm'], ff['deltav'], ff['deltab']  = sigma.calc_sigma(ff, num=8)
ff = ff.drop(['use_dens'], axis=1)

### MACH NUMBER
ff['MA'] = ff.vr / ff.vA
ff['MS'] = np.sqrt((ff.vr)**2 + (ff.vt)**2 + (ff.vn)**2)/con.speed_of_sound

### BETA
ff['beta'] = beta(np.array(ff.Tp)*u.eV, np.array(ff.Np)/(u.cm*u.cm*u.cm), np.array(ff.B)*u.nT).value

# ### MAGNETIC PRESSURE
ff['Pm'] = magnetic_pressure(np.array(ff.B)*u.nT).value

### PROTON PRESSURE
ff['Pp'] = thermal_pressure(np.array(ff.Tp)*u.eV, np.array(ff.Np)/u.cm**3).value

### ORBITER DF
orbiter = ff.copy()

In [ ]:
### ADD POSITION INFORMATION AND SAVE
orbiter['lon'] = solo_carrington.lon.value
orbiter['lat'] = solo_carrington.lat.value
orbiter['rAU'] = solo_carrington.radius.to(u.AU).value
orbiter['sslon'] = solo_at_source_surface.lon.value
orbiter['sslat'] = solo_at_source_surface.lat.value
orbiter['ssrAU'] = solo_at_source_surface.radius.to(u.AU).value
orbiter['NpR2'] = orbiter.Np * (orbiter.rAU ** 2)
orbiter['BrR2'] = orbiter.Br * (orbiter.rAU ** 2)
orbiter.to_csv(os.path.join(RES_DIR, 'orbiter.csv'))
orbiter